In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools

import npts

import time 
import copy

In [ ]:
# ## Load Data (Grid Load in Northern California)

# Data from 
# http://www.caiso.com/planning/Pages/ReliabilityRequirements/Default.aspx#Historical ,
# the webpage seems down, however.

# In[2]:

caiso_df = pd.read_excel("data/HistoricalEMSHourlyLoad_2014-2016.xlsx")
caiso_df.index = caiso_df.Dates

PGE_loads = caiso_df.PGE
#SCE_loads = caiso_df.SCE
PGE_loads.index.name = ''

In [ ]:
PGE_loads.plot()


# In[4]:

print('we have years:', set(PGE_loads.index.year))
data_used = PGE_loads[PGE_loads.index.year < 2016]
indep_test = PGE_loads[PGE_loads.index.year >= 2016]


# ## Fit and test the models 



In [ ]:
# In[5]:

indep_test_rmse = pd.DataFrame()
time_taken = pd.DataFrame()
model_objs = pd.DataFrame()

In [ ]:
MIN_REG = 1E-10 * np.abs(data_used).mean()
MAX_REG = MIN_REG * 1E10
MIN_REG, MAX_REG

In [ ]:
# In[15]:

models_modelnames = []#(npts.Baseline(), 'constant avg.', 1.)]

features = [npts.HourOfDay, npts.MonthOfYear, npts.DayOfWeek]#, npts.USHoliday]

for n_features in [2,3]:
    for features_used in itertools.combinations(features,n_features):
        models_modelnames.append((npts.Baseline(*(f(lambdas=[MIN_REG#1E-8
                                                            ]) for f in features_used)), 
                                  ' and '.join(f.__name__ for f in features_used) + ' avg.',
                                  1.))


models_modelnames.append((npts.Baseline(*(f(lambdas=np.logspace(np.log10(MIN_REG),#-8,
                                                                np.log10(MAX_REG),#-2, 
                                                                10)) for f in features), verbose=True), 
                          ' and '.join(f.__name__ for f in features) + ' bas.',
                          .75))
            

# features_plus_holiday = features + [npts.USHoliday]
# models_modelnames.append((npts.Baseline(*(f(lambdas=np.logspace(np.log10(MIN_REG),#-8,
#                                                             np.log10(MAX_REG),#-2, 
#                                                             10)) for f in features_plus_holiday), verbose=True), 
#                       ' and '.join(f.__name__ for f in features_plus_holiday) + ' bas.',
#                       .75))

In [ ]:
day_seconds = 86400
tropical_year_seconds = 365.24219 * day_seconds
week_seconds = 7 * day_seconds

models_modelnames.append(
    [npts.Harmonic([day_seconds, tropical_year_seconds, week_seconds]), 
                     'daily and annual and weekly harmonic', .75])

In [ ]:
# In[16]:

np.random.seed(0)


def sparsify_data(data, frac):
    return data[np.random.uniform(size=len(data)) < frac]

# for data, dataname in [(sparsify_data(data_used, target/len(data_used)), f'{target:e} observations')
#                        for target in (1E5, 1E4, 1E3, 1E2)]:
for data in [sparsify_data(data_used, target/len(data_used)) for target in (1E5, 1E4, 1E3, 1E2)]:
    dataname = f'M = {len(data)}'
    print(dataname)
#     print(len(data),dataname)
    for model, modelname, train_frac in models_modelnames:
        
        model_used = copy.copy(model)
        print(f'fitting {modelname} using {100*train_frac:.0f}% train data')
        model_objs.loc[dataname, modelname] = model_used
        
        s = time.time()
        model_used.fit(data,train_frac=train_frac)
        time_taken.loc[dataname, modelname] = time.time() - s
        pred = model_used.predict(indep_test.index)
        indep_test_rmse.loc[dataname, modelname] = np.sqrt(np.mean((indep_test - pred)**2)) 


In [ ]:
indep_test_rmse

In [ ]:
print(indep_test_rmse.to_latex(float_format='%.3f'))

In [ ]:
indep_test_rmse

In [ ]:
indep_test_rmse

In [ ]:
indep_test_rmse

In [ ]:
indep_test_rmse

In [ ]:
indep_test_rmse

In [ ]:
win_s = 0
win_e = win_s + 24*7

plt.figure(figsize=(14,5))
model_objs.iloc[0,4].predict(indep_test.index[win_s:win_e]).plot(label='regularized, 1.7E4 obs.', style='r--')
model_objs.iloc[-1,4].predict(indep_test.index[win_s:win_e]).plot(label='regularized, 1E2 obs.', style='b--')
indep_test[win_s:win_e].plot(label='real', style='k-')
plt.ylabel('MWh')

plt.legend()

plt.figure(figsize=(14,5))

model_objs.iloc[0,9].predict(indep_test.index[win_s:win_e]).plot(label='regularized + holiday, 1.7E4 obs.', style='r-.')
model_objs.iloc[-1,9].predict(indep_test.index[win_s:win_e]).plot(label='regularized + holiday, 1E2 obs.', style='b-.')
indep_test[win_s:win_e].plot(label='real', style='k-')
plt.ylabel('MWh')

plt.legend()

plt.figure(figsize=(14,5))

model_objs.iloc[0,-1].predict(indep_test.index[win_s:win_e]).plot(label='harmonic, 1.7E4 obs.', style='r-.')
model_objs.iloc[-1,-1].predict(indep_test.index[win_s:win_e]).plot(label='harmonic, 1E2 obs.', style='b-.')
indep_test[win_s:win_e].plot(label='real', style='k-')
plt.ylabel('MWh')

plt.legend()

